In [1]:
import nltk
import re
import string
import unicodedata
import pandas as pd
import numpy as np
import datetime as dt
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import os
from urllib.request import Request, urlopen

In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
validated_food_file_cleaned = pd.read_csv('validated_food_file_cleaned.csv')
food_category_df = pd.read_csv('parsed_food_descr_food_category.csv')

In [4]:
stopword_list = stopwords.words('english')
stopword_list = [re.sub('\W','',word.lower()) for word in stopword_list]

In [5]:
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

In [6]:
# For ignoring SSL certificate errors

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [7]:
# Input from user
url = 'https://genius.com/A-boogie-wit-da-hoodie-love-drugs-and-sex-lyrics'

In [8]:
# Making the website believe that you are accessing it using a mozilla browser
req = Request(url, headers = { 'User-Agent' : 'Mozilla/5.0' })
webpage = urlopen(req).read()

In [9]:
# Creating a BeautifulSoup object of the html page for easy extraction of data.

soup = BeautifulSoup(webpage, 'html.parser')
html = soup.prettify('utf-8')
song_json = {}
song_json["Lyrics"] = [];

In [10]:
#Extract Title of the song
for title in soup.findAll('title'):
  song_json["Title"] = title.text.strip()

In [11]:
song_json

{'Lyrics': [],
 'Title': 'A\xa0Boogie wit da Hoodie – Love Drugs and Sex Lyrics | Genius Lyrics'}

In [14]:
song = song_json["Title"].split('|')[0].split('–')[1].lower().replace('lyrics','').strip()

In [15]:
song_json["Title"].split('|')[0].split('–')[0].lower().strip()

'a\xa0boogie wit da hoodie'

In [16]:
# Extract the release date of the song
for span in soup.findAll('span', attrs = {'class': 'metadata_unit-info metadata_unit-info--text_only'}):
    print(span)
    song_json["Release date"] = span.text.strip()

<span class="metadata_unit-info metadata_unit-info--text_only">December 21, 2018</span>


In [17]:
# Extract the release date of the song
for span in soup.findAll('span', attrs = {'class': 'metadata_unit-info metadata_unit-info--text_only'}):
    print(span)
    song_json["Release date"] = span.text.strip()

<span class="metadata_unit-info metadata_unit-info--text_only">December 21, 2018</span>


In [18]:
len(soup.findAll('span', attrs = {'class': 'metadata_unit-info metadata_unit-info--text_only'}))

1

In [19]:
release_date = song_json["Release date"]

In [20]:
len(release_date)

17

In [21]:
release_date = song_json["Release date"]
#release_date_formatted = dt.datetime.strptime(release_date, '%B %d, %Y').strftime('%Y-%m-%d')
release_date_formatted = dt.datetime.strptime(release_date, '%Y').strftime('%Y-%m-%d')
release_year = dt.datetime.strptime(release_date, '%B %d, %Y').year
release_month = dt.datetime.strptime(release_date, '%B %d, %Y').month

ValueError: time data 'December 21, 2018' does not match format '%Y'

In [22]:
release_date_formatted

NameError: name 'release_date_formatted' is not defined

In [108]:
#Extract the Lyrics of the song
for div in soup.findAll('div', attrs = {'class': 'lyrics'}):
  song_json["Lyrics"].append(div.text.strip().split("\n"));

In [109]:
song_json["Lyrics"]

[['[Instrumental]']]

In [19]:
#filename = '/Users/michaellee/Desktop/python_lyrics_project/granata_lyrics/free_beer/loyal_to_the_leaves'
#lyrics_file = open(filename, 'r')
#lyrics_content = lyrics_file.read()
#lyrics_file.close()

In [20]:
artist_lyric_words = {}

In [21]:
fil = '/Users/michaellee/Desktop/python_lyrics_project/analyzed_song_urls.txt'
open_file = open(fil,'r')
open_file_lines = open_file.readlines()
open_file.close()

In [22]:
open_file_lines

['yeetyeetyeet']

In [23]:
open_file = open(fil,'a')
open_file.write('yeet')
open_file.close()

In [24]:
for line in song_json["Lyrics"][0]:
    if len(line) == 0:
        verse_artists_unique = []
        continue
    if line[0] == '[':
        verse_artists_unique = []
        line = line.strip('[]')
        line = line.split(':')
        if len(line) == 1:
            verse_artists = song_json["Title"].split('|')[0].split('–')[0].lower().strip()
            verse_artists = verse_artists.replace('tyler, the creator','tyler the creator')
            verse_artists_unique.append(verse_artists)
            continue
        else:
            verse_artists = line[1]
            verse_artists = verse_artists.lower()
            verse_artists = strip_accents(verse_artists)
            verse_artists = verse_artists.replace('both','')
            verse_artists = verse_artists.replace('tyler, the creator','tyler the creator')
            verse_artists = verse_artists.split(',')
            for artists in verse_artists:
                artists = artists.replace('(','')
                artists = artists.replace(')','')
                if '&' in artists:
                    artists = artists.split('&')
                if '+' in artists:
                    artists = artists.split('+')
                if type(artists) == str:
                    artists = artists.strip()
                    verse_artists_unique.append(artists)
                if type(artists) == list:
                    for artist in artists:
                        artist = artist.strip()
                        verse_artists_unique.append(artist)
            verse_artists_unique = list(set(verse_artists_unique))
            continue
    
    words = line.split()
    for word in words:
        word = re.sub('\W','',word.lower())
        word = lemmatizer.lemmatize(word)
        word = strip_accents(word)
        for artist in verse_artists_unique:
            if artist in artist_lyric_words:
                artist_lyric_words[artist].append(word)
            else:
                artist_lyric_words[artist] = []
                artist_lyric_words[artist].append(word)

In [25]:
artist_lyric_words_count = {}
artist_lyric_words_count_excl_stopwords = {}

In [26]:
for artist in artist_lyric_words.keys():
    artist_lyric_words_count[artist] = len(artist_lyric_words[artist])
    artist_lyric_words_count_excl_stopwords[artist] = len([word for word in artist_lyric_words[artist] if word not in stopword_list])
        

In [27]:
artist_lyric_words_count

{'frank ocean': 320, 'tyler the creator': 399}

In [28]:
artist_lyric_words_count_excl_stopwords

{'frank ocean': 161, 'tyler the creator': 198}

In [29]:
artist_lyrics_content_one_word_dict = {}
artist_lyrics_content_two_word_dict = {}

In [30]:
for artist in artist_lyric_words.keys():  
    for word_num in range(artist_lyric_words_count[artist]-1):
        if word_num < artist_lyric_words_count[artist]:
            one_word = artist_lyric_words[artist][word_num]
            if artist in artist_lyrics_content_one_word_dict:
                if one_word in artist_lyrics_content_one_word_dict[artist]:
                    artist_lyrics_content_one_word_dict[artist][one_word] += 1
                else:
                    artist_lyrics_content_one_word_dict[artist][one_word] = 1
            else:
                artist_lyrics_content_one_word_dict[artist] = {}
                artist_lyrics_content_one_word_dict[artist][one_word] = 1

            two_word = artist_lyric_words[artist][word_num] + ' ' + artist_lyric_words[artist][word_num+1]
            if artist in artist_lyrics_content_two_word_dict:
                if two_word in artist_lyrics_content_two_word_dict[artist]:
                    artist_lyrics_content_two_word_dict[artist][two_word] += 1
                else:
                    artist_lyrics_content_two_word_dict[artist][two_word] = 1
            else:
                artist_lyrics_content_two_word_dict[artist] = {}
                artist_lyrics_content_two_word_dict[artist][two_word] = 1

In [31]:
#artist_lyrics_content_one_word_dict

In [32]:
#artist_lyrics_content_two_word_dict

In [33]:
artist_lyric_word_count_df = pd.DataFrame(columns=['word',
                                                  'word_count',
                                                  'artist'])

In [34]:
for artist in artist_lyrics_content_one_word_dict.keys():
    artist_lyrics_content_one_word_df = pd.DataFrame.from_dict(artist_lyrics_content_one_word_dict[artist], orient='index')
    artist_lyrics_content_one_word_df = artist_lyrics_content_one_word_df.reset_index()
    artist_lyrics_content_one_word_df['artist'] = artist
    artist_lyrics_content_one_word_df.columns = ['word','word_count','artist']
    artist_lyric_word_count_df = pd.concat([artist_lyric_word_count_df,artist_lyrics_content_one_word_df],axis=0,ignore_index=True)
    

In [35]:
for artist in artist_lyrics_content_two_word_dict.keys():
    artist_lyrics_content_two_word_df = pd.DataFrame.from_dict(artist_lyrics_content_two_word_dict[artist], orient='index')
    artist_lyrics_content_two_word_df = artist_lyrics_content_two_word_df.reset_index()
    artist_lyrics_content_two_word_df['artist'] = artist
    artist_lyrics_content_two_word_df.columns = ['word','word_count','artist']
    artist_lyric_word_count_df = pd.concat([artist_lyric_word_count_df,artist_lyrics_content_two_word_df],axis=0,ignore_index=True)
    

In [36]:
artist_food_words_df = pd.merge(artist_lyric_word_count_df,validated_food_file_cleaned[['food_description','food_word_net_effect_value']],how='inner',left_on='word',right_on='food_description')
artist_food_words_df = artist_food_words_df.drop(labels='food_description',axis=1)
artist_food_words_df = artist_food_words_df.rename(columns={'food_word_net_effect_value':'food_word_value'})
artist_food_words_df['food_word_total'] = artist_food_words_df['word_count'] * artist_food_words_df['food_word_value']
artist_food_words_df = artist_food_words_df.rename(columns={'word':'food_word',
                                                           'word_count':'food_word_count'})
artist_food_words_df['song'] = song
artist_food_words_df['release_date'] = release_date_formatted
artist_food_words_df['release_year'] = release_year
artist_food_words_df['release_month'] = release_month

artist_food_words_df = pd.merge(artist_food_words_df,food_category_df,how='left',left_on='food_word',right_on='food_description')
artist_food_words_df = artist_food_words_df.drop(labels=['food_description','food_category_id'],axis=1)


In [37]:
artist_food_words_df

,food_word,food_word_count,artist,food_word_value,food_word_total,song,release_date,release_year,release_month,food_category_description
0,nut,1,frank ocean,1,1,she,2011-05-10,2011,5,Nut and Seed Products


In [973]:
artist_food_words_count = {}
artist_food_words_perc_all_words = {}
artist_food_words_perc_all_words_excl_sw = {}

for artist in list(artist_lyric_words_count.keys()):
    artist_food_words_count[artist] = sum(artist_food_words_df.loc[artist_food_words_df.artist == artist]['food_word_total'])
    artist_food_words_perc_all_words[artist] = artist_food_words_count[artist] / artist_lyric_words_count[artist]
    artist_food_words_perc_all_words_excl_sw[artist] = artist_food_words_count[artist] / artist_lyric_words_count_excl_stopwords[artist]

In [974]:
artist_food_words_count

{'a\xa0tribe called quest': 0, 'q-tip': 2}

In [975]:
artist_lyric_words_df = pd.DataFrame.from_dict(artist_lyric_words_count,orient='index')
artist_lyric_words_df.columns = ['lyrics_word_count']
artist_lyric_words_excl_sw_df = pd.DataFrame.from_dict(artist_lyric_words_count_excl_stopwords,orient='index')
artist_lyric_words_excl_sw_df.columns = ['lyrics_word_count_excl_sw']
artist_food_words_count_df = pd.DataFrame.from_dict(artist_food_words_count,orient='index')
artist_food_words_count_df.columns = ['food_words_count']
artist_food_words_perc_all_words_df = pd.DataFrame.from_dict(artist_food_words_perc_all_words,orient='index')
artist_food_words_perc_all_words_df.columns = ['food_words_perc_all_lyric_words']
artist_food_words_perc_all_words_excl_sw_df = pd.DataFrame.from_dict(artist_food_words_perc_all_words_excl_sw,orient='index')
artist_food_words_perc_all_words_excl_sw_df.columns = ['food_words_perc_lyric_words_excl_sw']

song_agg_stats = pd.concat([artist_lyric_words_df,
                            artist_lyric_words_excl_sw_df,
                            artist_food_words_count_df,
                            artist_food_words_perc_all_words_df,
                           artist_food_words_perc_all_words_excl_sw_df],axis=1)
song_agg_stats = song_agg_stats.replace(np.nan,0)
song_agg_stats = song_agg_stats.reset_index()
song_agg_stats = song_agg_stats.rename(columns={'index':'artist'})
song_agg_stats['song'] = song
song_agg_stats['release_date'] = release_date_formatted
song_agg_stats['release_year'] = release_year
song_agg_stats['release_month'] = release_month



In [976]:
song_agg_stats

,artist,lyrics_word_count,lyrics_word_count_excl_sw,food_words_count,food_words_perc_all_lyric_words,food_words_perc_lyric_words_excl_sw,song,release_date,release_year,release_month
0,a tribe called quest,53,28,0,0.00000,0.000000,sucka nigga,1993-11-09,1993,11
1,q-tip,542,288,2,0.00369,0.006944,sucka nigga,1993-11-09,1993,11


In [519]:
#artist_agg_stats_df = pd.DataFrame(columns=['artist',
#                                          'song',
#                                          'release_date',
#                                          'release_year',
#                                          'release_month',
#                                          'lyrics_word_count',
#                                          'lyrics_word_count_excl_sw',
#                                         'food_words_count',
#                                         'food_words_perc_all_words',
#                                         'food_words_perc_excl_sw'])
#artist_food_word_stats_df = pd.DataFrame(columns=['food_word',
#                                               'food_word_count',
#                                               'artist',
#                                               'song',
#                                               'release_date',
#                                               'release_year',
#                                               'release_month',
#                                               'food_category_description'])

In [520]:
artist_agg_stats_df = pd.concat([artist_agg_stats_df,song_agg_stats_df],axis=0)
artist_food_word_stats_df = pd.concat([artist_food_word_stats_df,song_food_word_stats_df],axis=0)

In [521]:
artist_agg_stats_df

,artist,song,release_date,release_year,release_month,lyrics_word_count,lyrics_word_count_excl_sw,food_words_count,food_words_perc_all_words,food_words_perc_excl_sw
0,j. cole,for whom the bell tolls,2016-12-09,2016,12,180,99,0,0.000000,0.000000
0,j. cole,foldin clothes,2016-12-09,2016,12,553,295,9,0.016275,0.030508
0,j. cole,immortal,2016-12-09,2016,12,520,298,3,0.005769,0.010067
0,j. cole,deja vu,2016-12-09,2016,12,830,444,2,0.002410,0.004505
0,j. cole,ville mentality,2016-12-08,2016,12,337,177,0,0.000000,0.000000


In [522]:
artist_food_word_stats_df

,food_word,food_word_count,artist,song,release_date,release_year,release_month,food_category_description
0,breakfast,1,j. cole,foldin clothes,2016-12-09,2016,12,Baked Products
1,raisin,1,j. cole,foldin clothes,2016-12-09,2016,12,Baked Products
2,bran,1,j. cole,foldin clothes,2016-12-09,2016,12,NaN
3,banana,1,j. cole,foldin clothes,2016-12-09,2016,12,Fruits and Fruit Juices
4,almond,2,j. cole,foldin clothes,2016-12-09,2016,12,Nut and Seed Products
5,milk,2,j. cole,foldin clothes,2016-12-09,2016,12,Dairy and Egg Products
6,drinking,1,j. cole,foldin clothes,2016-12-09,2016,12,Beverages
7,raisin bran,1,j. cole,foldin clothes,2016-12-09,2016,12,NaN
8,almond milk,2,j. cole,foldin clothes,2016-12-09,2016,12,Beverages
0,cream,1,j. cole,immortal,2016-12-09,2016,12,Dairy and Egg Products
